<a href="https://colab.research.google.com/github/entelecheia/378A/blob/main/notebooks/03/03_dart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup to persist data in Google Drive

In [1]:
import os, sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
      drive.mount("/content/drive")
      !ln -s "/content/drive/My Drive/colab_workspace" workspace
    WORKSPACE_DIR = "/content/workspace"
else:
    WORKSPACE_DIR = "../../workspace" 
print(f'WORKSPACE_DIR = {WORKSPACE_DIR}')
data_dir = os.path.join(WORKSPACE_DIR, "data")
os.makedirs(data_dir, exist_ok=True)

Mounted at /content/drive
WORKSPACE_DIR = /content/workspace


In [54]:
# %%capture
%pip install opendartreader
%pip install python-dotenv

In [9]:
from dotenv import load_dotenv, find_dotenv
# find .env automagically by walking up directories until it's found
dotenv_path = WORKSPACE_DIR + '/.env'
# load up the entries as environment variables
load_dotenv(dotenv_path)
DART_API_KEY = os.environ.get("DART_API_KEY")

### Download and Extract Company Disclosure from DART

In [13]:
import OpenDartReader

dart = OpenDartReader(DART_API_KEY) 

```python
# == 1. 공시정보 검색 ==
# 삼성전자 2019-07-01 하루 동안 공시 목록 (날짜에 다양한 포맷이 가능합니다)
dart.list('005930', end='2019-7-1')

# 삼성전자 상장이후 모든 공시 목록 (5,142 건+)
dart.list('005930', start='1900') 

# 삼성전자 2010-01-01 ~ 2019-12-31 모든 공시 목록 (2,676 건)
dart.list('005930', start='2010-01-01', end='2019-12-31') 

# 삼성전자 1999-01-01 이후 모든 정기보고서
dart.list('005930', start='1999-01-01', kind='A', final=False)

# 삼성전자 1999년~2019년 모든 정기보고서(최종보고서)
dart.list('005930', start='1999-01-01', end='2019-12-31', kind='A') 


# 2020-07-01 하루동안 모든 공시목록
dart.list(end='20200701')

# 2020-01-01 ~ 2020-01-10 모든 회사의 모든 공시목록 (4,209 건)
dart.list(start='2020-01-01', end='2020-01-10')

# 2020-01-01 ~ 2020-01-10 모든 회사의 모든 공시목록 (정정된 공시포함) (4,876 건)
dart.list(start='2020-01-01', end='2020-01-10', final=False)

# 2020-07-01 부터 현재까지 모든 회사의 정기보고서
dart.list(start='2020-07-01', kind='A')

# 2019-01-01 ~ 2019-03-31 모든 회사의 정기보고서 (961건)
dart.list(start='20190101', end='20190331', kind='A')

# 기업의 개황정보
dart.company('005930')

# 회사명에 삼성전자가 포함된 회사들에 대한 개황정보
dart.company_by_name('삼성전자')

# 삼성전자 사업보고서 (2018.12) 원문 텍스트
xml_text = dart.document('20190401004781')


# ==== 2. 사업보고서 ====
# 삼성전자(005930), 배당관련 사항, 2018년
dart.report('005930', '배당', 2018) 

# 서울반도체(046890), 최대주주 관한 사항, 2018년
dart.report('046890', '최대주주', 2018) 

# 서울반도체(046890), 임원 관한 사항, 2018년
dart.report('046890', '임원', 2018) 

# 삼성바이오로직스(207940), 2019년, 소액주주에 관한 사항
dart.report('207940', '소액주주', '2019')


# ==== 3. 상장기업 재무정보 ====
# 삼성전자 2018 재무제표 
dart.finstate('삼성전자', 2018) # 사업보고서

# 삼성전자 2018Q1 재무제표
dart.finstate('삼성전자', 2018, reprt_code='11013')

# 여러종목 한번에
dart.finstate('00126380,00164779,00164742', 2018)
dart.finstate('005930, 000660, 005380', 2018)
dart.finstate('삼성전자, SK하이닉스, 현대자동차', 2018)

# 단일기업 전체 재무제표 (삼성전자 2018 전체 재무제표)
dart.finstate_all('005930', 2018)

# 재무제표 XBRL 원본 파일 저장 (삼성전자 2018 사업보고서)
dart.finstate_xml('20190401004781', save_as='삼성전자_2018_사업보고서_XBRL.zip')

# XBRL 표준계정과목체계(계정과목)
dart.xbrl_taxonomy('BS1')


# ==== 4. 지분공시 ====
# 대량보유 상황보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders('삼성전자')

# 임원ㆍ주요주주 소유보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders_exec('005930')


# ==== 5. 확장 기능 ====
# 지정한 날짜의 공시목록 전체 (시간 정보 포함)
dart.list_date_ex('2020-01-03')

# 개별 문서 제목과 URL
rcp_no = '20190401004781' # 삼성전자 2018년 사업보고서
dart.sub_docs(rcp_no)

# 제목이 잘 매치되는 순서로 소트
dart.sub_docs('20190401004781', match='사업의 내용')

# 첨부 문서 제목과 URL
dart.attach_doc_list(rcp_no)

# 제목이 잘 매치되는 순서로 소트
dart.attach_doc_list(rcp_no, match='감사보고서')

# 첨부 파일 제목과 URL
dart.attach_file_list(rcp_no)
```

In [15]:
# 삼성전자 1999-01-01 이후 모든 정기보고서
dart.list('005930', start='1999-01-01', kind='A')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,사업보고서 (2021.12),20220308000798,삼성전자,20220308,연
1,00126380,삼성전자,005930,Y,분기보고서 (2021.09),20211115001965,삼성전자,20211115,
2,00126380,삼성전자,005930,Y,반기보고서 (2021.06),20210817001416,삼성전자,20210817,
3,00126380,삼성전자,005930,Y,분기보고서 (2021.03),20210517001185,삼성전자,20210517,
4,00126380,삼성전자,005930,Y,사업보고서 (2020.12),20210309000744,삼성전자,20210309,연
...,...,...,...,...,...,...,...,...,...
86,00126380,삼성전자,005930,Y,반기보고서 (2000.06),20000814000482,삼성전자,20000814,
87,00126380,삼성전자,005930,Y,[첨부추가]사업보고서 (1999.12),20000330000796,삼성전자,20000330,연
88,00126380,삼성전자,005930,Y,분기보고서 (2000.03),20000515000739,삼성전자,20000515,
89,00126380,삼성전자,005930,Y,[기재정정]반기보고서 (1999.06),19990824000003,삼성전자,19990824,


In [50]:
# 제목이 잘 매치되는 순서로 소트
sub_docs = dart.sub_docs('20220308000798', match='이사의 경영진단 및 분석의견')
sub_docs

,title,url
5,IV. 이사의 경영진단 및 분석의견,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1,【 대표이사 등의 확인 】,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
0,사 업 보 고 서,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
6,V. 회계감사인의 감사의견 등,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
7,VI. 이사회 등 회사의 기관에 관한 사항,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2,I. 회사의 개요,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
3,II. 사업의 내용,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
14,【 전문가의 확인 】,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
9,VIII. 임원 및 직원 등에 관한 사항,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
4,III. 재무에 관한 사항,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [51]:
url = sub_docs.url[sub_docs.index[0]]

In [52]:
import requests
from bs4 import BeautifulSoup
        
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [1]:
#soup.get_text()